In [1]:
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import matplotlib.pyplot as plt
import numpy as np
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from numpy.linalg import norm

In [2]:
def draw_landmarks_on_image(rgb_image, detection_result):
  pose_landmarks_list = detection_result.pose_landmarks
  annotated_image = np.copy(rgb_image)

  # Loop through the detected poses to visualize.
  for idx in range(len(pose_landmarks_list)):
    pose_landmarks = pose_landmarks_list[idx]

    # Draw the pose landmarks.
    pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    pose_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in pose_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      pose_landmarks_proto,
      solutions.pose.POSE_CONNECTIONS,
      solutions.drawing_styles.get_default_pose_landmarks_style())
  return annotated_image

In [3]:
def convert_landmarks_to_numpy(pose_landmarks):
    # Extract the pose_landmarks list from the PoseLandmarkerResult object
    # Each pose_landmark is an array of NormalizedLandmark objects
    landmarks_list = pose_landmarks.pose_landmarks[0]

    # Extract the x, y, z values from each NormalizedLandmark and create a list of them
    normalized_landmarks = [[landmark.x, landmark.y, landmark.z] for landmark in landmarks_list]

    # Convert the list of landmarks to a NumPy array
    return np.array(normalized_landmarks)

In [4]:
def file_display(name):
    plt.figure()
    img = cv2.imread(name)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

In [5]:
def file_process(name, detector):
    image = mp.Image.create_from_file(name)
    detection_result = detector.detect(image)
    array = convert_landmarks_to_numpy(detection_result).flatten()

    plt.figure()
    annotated_image = draw_landmarks_on_image(image.numpy_view(), detection_result)
    plt.imshow(cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB))

    return array

In [6]:
def process(file1, file2):
    file_display(file1)
    file_display(file2)
    plt.show()

    # Create an PoseLandmarker object.
    base_options = python.BaseOptions(model_asset_path='pose_landmarker.task')
    options = vision.PoseLandmarkerOptions(
        base_options=base_options,
        output_segmentation_masks=True)
    detector = vision.PoseLandmarker.create_from_options(options)

    res1 = file_process(file1, detector)
    res2 = file_process(file2, detector)

    cosine = np.dot(res1, res2)/(norm(res1) * norm(res2))
    print("Similarity:", cosine)

    plt.show()

In [7]:
file1 = input("File 1:") + ".jpg"
file2 = input("File 2:") + ".jpg"
process(file1, file2)

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


<Figure size 640x480 with 0 Axes>